In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
from data_loader import load_sector_data, get_market_color

In [7]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [8]:
# 업종별 점유율을 시각화 하는 함수

def visualize_sector_share(market=None, month_ago=3):
  
    df = load_sector_data(market, month_ago=month_ago)
  
    # 업종별 카운트 계산
    df_sector_cnt = df['Sector'].value_counts().sort_values(ascending=True)
    
    # 시장의 총 종목 수 계산
    total_count = len(df)  # 전체 종목 수
    
    # 업종별 비율 계산
    sector_ratio = df_sector_cnt / total_count * 100
    
    # Figure의 세로 사이즈 조정 (업종수에 따라)
    figure_height = 400 + (len(df_sector_cnt) * 30)  
    
    # 시장별 그래프 색상 설정
    color = get_market_color(market)
    
    # Plotly로 그래프 작성
    fig = px.bar(
        x=sector_ratio,
        y=df_sector_cnt.index,
        color_discrete_sequence=[color],
        title=f'{market} 시장 업종별 점유율',
        labels={'x':'점유율 (%)','y':'업종명'},
        height=figure_height
    )  
    fig.show()
    
    display(pd.DataFrame(df['Sector'].value_counts().sort_values(ascending=False)))
    

업종별 수익률 => 색상으로 위험도를 구분 -> 위험도 순으로 정렬 -> 이를 통해 위험도 대비 수익률

In [9]:
# 업종별 수익률 평균을 계산하고 시각화하는 함수

# plotly를 사용해 사용자에게 좀 더 유용한 정보를 제공

def avg_return_by_sector(market=None, month_ago=3, risk=None):
    
    df = load_sector_data(market, month_ago=month_ago)  # 데이터 로드
    
    # 시장의 평균 수익률 계산
    market_avg_return = df['TotalReturn'].mean()
    
    # 업종별 평균 수익률, 평균 위험도 계산
    mean_return_by_sector = df.groupby('Sector').agg({'TotalReturn': 'mean', 'Risk':'mean'}) \
                            .reset_index().sort_values(by='Risk', ascending=True)
    
    # 위험도 구간 필터링 => 수익률 내림차순
    if risk is not None:
        mean_return_by_sector = mean_return_by_sector[mean_return_by_sector['Risk'] <= risk]\
                                .sort_values(by='TotalReturn')

    # Figure의 세로 사이즈 조정 (업종수에 따라)
    figure_height = 400 + (len(mean_return_by_sector) * 30)        
    
    # Plotly로 그래프 작성
    fig = px.bar(
        mean_return_by_sector,
        x='TotalReturn', 
        y='Sector',
        color='Risk',  # 위험도를 색상으로 사용
        color_continuous_scale=px.colors.sequential.Reds,  # 색상 스케일 설정
        title=f'{market} 업종별 {month_ago}개월 평균 수익률',
        labels={'x':'평균 수익률 (%)','y':'업종명'},
        height=figure_height
    )
    
    # 평균 수직선 추가
    fig.add_shape(type='line', xref='x', yref='y',
                  x0=market_avg_return, y0=-0.5, x1=market_avg_return, y1=len(mean_return_by_sector)-0.5,
                  line=dict(color='blue', dash='dash'),
                  name=f'{market} 평균 수익률(%)')
    
    # 평균 라벨 추가
    fig.add_annotation(
        x=market_avg_return,
        y=len(mean_return_by_sector) - 0.5,
        text=f'{market} 평균 수익률(%) : {market_avg_return:.3f} %',
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40,
        font=dict(color='blue')
    )
    
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=20, color='black', family='Arial')
    ) 
    
    fig.show()

In [12]:
avg_return_by_sector(market='KOSPI', month_ago=1, risk=3)

1개월 간의 data를 불러옵니다.


In [11]:
avg_return_by_sector('KOSDAQ',month_ago=1, risk=2)

1개월 간의 data를 불러옵니다.


In [12]:
avg_return_by_sector('ETF',month_ago=1, risk=2)

1개월 간의 data를 불러옵니다.


In [20]:
visualize_sector_share('KOSPI')

3개월 간의 data를 불러옵니다.


,count
Sector,
화학,73
자동차부품,73
제약,62
식품,54
"섬유,의류,신발,호화품",49
건설,42
철강,37
증권,34
건축자재,31


In [18]:
visualize_sector_share('KOSDAQ')

3개월 간의 data를 불러옵니다.


,count
Sector,
반도체와반도체장비,142
창업투자,119
제약,111
IT서비스,82
전자장비와기기,78
자동차부품,75
건강관리장비와용품,74
기계,73
소프트웨어,72


In [17]:
visualize_sector_share('ETF')

3개월 간의 data를 불러옵니다.


,count
Sector,
해외주식,295
국내 업종/테마,248
채권,161
국내 시장지수,88
기타,79
국내파생,40
원자재,18
